In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier  # 随机森林
from sklearn.model_selection import GridSearchCV, train_test_split # 网格评分器

from sklearn.metrics import make_scorer # 创建评分
from sklearn.metrics import f1_score # f1 评分

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer # 文本编码

# scorer = make_scorer(f1_score)
# parameters = {'kernel':['poly', 'rbf'],'C':[0.1, 1, 10]}

train_data = pd.read_csv('census.csv') # 训练数据
pred_data = pd.read_csv('test_census.csv') # 需要预测的数据

train_data['income'] = np.where(train_data['income'] == '<=50K', 0, 1)

y_len = train_data.shape[1]

x = train_data.iloc[:, 0:y_len-1]
y = train_data.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


def data_preprocessing(df):
    '''
    数据预处理
    '''
    
    strip = ['workclass', 'education_level', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
#     one_hot_encoder = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
    one_hot_encoder = ['workclass']
    label_encoder = ['education_level']
    
    # 去左右空格
    for col in strip:
        df[col] = df[col].str.strip()
    
    # label_encoder 编码
#     for col in label_encoder:
#         df[col] = LabelEncoder().fit(df[col]).transform(df[col])
    
    # 热编码 
    for col in one_hot_encoder:
        if col == 'native-country':
            df[col] = df[col].where(df[col].isin(['United-States', 'Mexico']), 'Other')
        
        encoder = LabelBinarizer()
        lb = pd.DataFrame(encoder.fit_transform(df[col]), columns=encoder.classes_, index=df.index)
        df = df.join(lb)
#     new = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
 
    #去掉原始特征
    
    return df

In [2]:
pred_data.columns.values
x_test.columns.values

list(set(x_test.columns.values) ^ set(pred_data.columns.values))

['Unnamed: 0']

In [4]:
aaa = pred_data.iloc[:, 1:pred_data.shape[1]]
aaa

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,21.0,Private,10th,6.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States
1,49.0,Private,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Wife,White,Female,0.0,0.0,40.0,United-States
2,44.0,Self-emp-not-inc,Assoc-acdm,12.0,Married-civ-spouse,Other-service,Wife,White,Female,0.0,0.0,99.0,United-States
3,34.0,Private,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,7298.0,0.0,46.0,United-States
4,24.0,Private,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,17.0,Private,11th,7.0,Never-married,Other-service,Own-child,White,Male,0.0,0.0,15.0,United-States
45218,50.0,Federal-gov,Some-college,10.0,Never-married,Adm-clerical,Not-in-family,White,Male,0.0,0.0,40.0,United-States
45219,17.0,Self-emp-inc,12th,8.0,Never-married,Adm-clerical,Own-child,White,Female,0.0,0.0,16.0,United-States
45220,38.0,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,7688.0,0.0,40.0,United-States


In [5]:
# data_preprocessing(x_test)
# aaa['education_level'].value_counts()
data_preprocessing(aaa)

TypeError: '<' not supported between instances of 'str' and 'float'

In [9]:
col = 'native-country'
x_train[col] = x_train[col].str.strip()
value_counts = x_train[col].value_counts(normalize=True)
value_counts.keys()
# keys = value_counts[value_counts>0.01].keys()
# list(keys)

# x_train['native-country'] = x_train['native-country'].where(x_train['native-country'].isin(keys), 'other')
# pd.get_dummies(x_train['native-country'], prefix='native-country')


# x_train['native-country'] = x_train['native-country'].where(x_train['native-country'].isin(value_counts[value_counts>0.01].keys()), 'other')

# x_train

# pd.get_dummies(x_train['native-country'], prefix='native-country', columns=[' China'])


# value_counts[value_counts>0.01]
# list(value_counts.where(value_counts > 0.01))

# x_train['native-country'].isin([' United-States'])

# x_train['native-country'] = x_train['native-country'].where(x_train['native-country'].isin([' United-States']), 'o')

# x_train['native-country'].value_counts(normalize=True)
# # x_train['native-country'][x_train['native-country'].isin([' United-States']), 'o']

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['United-States', 'Mexico', 'Philippines', 'Germany', 'Puerto-Rico',
       'Canada', 'El-Salvador', 'India', 'Cuba', 'China', 'England', 'Italy',
       'Dominican-Republic', 'South', 'Japan', 'Jamaica', 'Guatemala',
       'Columbia', 'Poland', 'Vietnam', 'Portugal', 'Haiti', 'Peru', 'Iran',
       'Taiwan', 'Nicaragua', 'Greece', 'Ecuador', 'France', 'Thailand',
       'Ireland', 'Cambodia', 'Hong', 'Outlying-US(Guam-USVI-etc)', 'Scotland',
       'Trinadad&Tobago', 'Laos', 'Yugoslavia', 'Hungary', 'Honduras',
       'Holand-Netherlands'],
      dtype='object')

In [2]:
def data_preprocessing(x):
    '''
    数据预处理
    '''
    
    strip = ['workclass', 'education_level', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
    one_hot_encoder = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
    label_encoder = ['education_level']
    
    # 去左右空格
    for col in strip:
        x[col] = x[col].str.strip()
    
    # LabelEncoder 编码    
    for col in label_encoder:
        x[col] = LabelEncoder().fit(x[col]).transform(x[col])
            
    
            
    return x

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
set(list(x_test.columns.values)) - set(list(x_train.columns.values))
list(set(x_train.columns.values) ^ set(x_test.columns.values))

['native-country_Holand-Netherlands']

In [18]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

/data/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,age,education_level,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
28962,27,9,13.0,0.0,0.0,37.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
27175,29,15,10.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
26140,41,11,9.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2377,39,12,14.0,8614.0,0.0,52.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
10490,49,11,9.0,0.0,0.0,40.0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21329,53,15,10.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
9772,40,9,13.0,0.0,0.0,48.0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
30115,50,9,13.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
30816,44,15,10.0,0.0,0.0,40.0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [15]:
y_red = model.predict(x_test)

ValueError: Number of features of the model must match the input. Model n_features is 170 and input n_features is 168 